### <span style="color: rgb(46, 204, 113);">1. 환경 설정 및 패키지 설치</span>
- 가상환경 만든 뒤, pip install ipykernel 실행
- 아래의 패키지 설치

In [1]:
%pip install -q python-dotenv langchain langchain-openai langchain-pinecone pinecone pandas langchain-community docx2txt langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q langchain_ollama 

Note: you may need to restart the kernel to use updated packages.


### <span style="color: rgb(46, 204, 113);">2. 환경 설정(환경변수, 시스템파라미터변수)</span>

In [3]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="e:/.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL="text-embedding-3-large" # small 버전은 1536차원, upstage는 4096

PINECONE_INDEX_NAME="better-reg-index"
PINECONE_INDEX_DIMENSION = 3072
PINECONE_INDEX_METRIC="cosine"
PINECONE_INDEX_REGION="us-east-1"
PINECONE_INDEX_CLOUD="aws"

### <span style="color: rgb(46, 204, 113);">3. 문서를 chunk로 분할하기</span>

In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# loader = Docx2txtLoader('./data/with_markdown-sample.docx')
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
# documents = loader.load_and_split(text_splitter) ## 이렇게 "\n"단위로 chunk를 안 나누고

loader = Docx2txtLoader('./tax_docs/with_markdown-sample.docx')
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500,
  chunk_overlap=200,
  separators=["\n\n", "\n", "제", "조", ".", " "]
)
documents = text_splitter.split_documents(document)
print(f"총 {len(documents)}개 청크 생성")

총 64개 청크 생성


### <span style="color: rgb(46, 204, 113);">4. metadata 추가하기</span>

In [7]:
import re

def remove_special_chars(text: str) -> str:
    """특수문자 및 \n제거(:는 그대로)"""
    # \n 제거
    text = text.replace("\n", " ")
    # 한글, 영문, 숫자, 공백, 마침표, 콤마만 남기고 전부 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s.,:]', '', text)
    # 불필요한 중복 공백 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned


# 사용 예시
content = "**소득세 납세 의무**: 거주자 및 국내원천소득자, 원천징수 대상 개인 및 법인 포함."
print(remove_special_chars(content))

소득세 납세 의무: 거주자 및 국내원천소득자, 원천징수 대상 개인 및 법인 포함.


In [8]:
from langchain_community.chat_models import ChatOllama
def extract_title_with_llm(content):
  """Exaone을 사용해서 제목 추출"""
  title_extractor_llm = ChatOllama(
                model="exaone3.5:2.4b",
                temperature=0.1, # 0에 가까우면 답변이 일관적
                num_predict=30 # 최대 토근 30토큰까지만 출력
            )
  prompt = f'''다음 소득세법 조문의 핵심 제목을 30토큰 이내로 간단히 완벽하게 말이 되도록 추출해 주세요. 중간에 말이 끊기면 안 되요.
  예 : "소득세 납세의무 범위 : 공동사업자별 소득과세, 상속과세, 증여자"
  조문 : {content}'''
  ai_message = title_extractor_llm.invoke(prompt)
  title = ai_message.content.strip()
  return remove_special_chars(title)
content = """제5조(과세기간) ① 소득세의 과세기간은 1월 1일부터 12월 31일까지 1년으로 한다.
② 거주자가 사망한 경우의 과세기간은 1월 1일부터 사망한 날까지로 한다.
③ 거주자가 주소 또는 거소를 국외로 이전(이하 “출국”이라 한다)하여 비거주자가 되는 경우의 과세기간은 1월 1일부터 출국한 날까지로 한다.
"""
print(extract_title_with_llm(content))

C:\Users\Admin\AppData\Local\Temp\ipykernel_10404\3252744802.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  title_extractor_llm = ChatOllama(


소득세 과세기간 : 연 1월12월, 사망 시 사망일까지, 국외 이전 시 출국일까지


In [9]:
def categorize_content(content):
    """내용 카테고리 분류"""
    # 쳇 gpt가 meta 데이터에 쓴다고 하니 만들어 준 카테고리
    categories = {
    '납세의무': ['납세의무', '거주자', '비거주자', '원천징수', '공동사업자', '상속인', '증여자', '신탁재산'],
    '세율계산': ['세율', '과세표준', '산출세액', '결정세액', '종합소득', '퇴직소득'],
    '근로소득': ['근로소득', '총급여', '급여', '연봉', '임금', '퇴직소득'],
    '사업소득': ['사업소득', '공동사업', '주택임대소득', '부업소득'],
    '이자배당': ['이자소득', '배당소득', '예금이자', '채권', '의제배당'],
    '양도소득': ['양도소득', '자산양도', '부동산양도', '주식양도'],
    '연금소득': ['연금소득', '공적연금', '사적연금', '연금보험'],
    '기타소득': ['기타소득', '상금', '보상금', '발명보상금', '종교인소득'],
    '공제감면': ['공제', '소득공제', '세액공제', '기본공제', '감면'],
    '비과세': ['비과세', '면제', '복무급여', '실업급여', '출산휴가급여', '장학금'],
    '신고납부': ['신고', '납부', '납세지', '신고기한', '원천징수'],
    '과세기간': ['과세기간', '과세연도', '사업연도'],
    '과세소득구분': ['종합소득', '퇴직소득', '양도소득', '금융투자소득'],
    }
    # categories = []
    for category, keywords in categories.items():
        if any(keyword in content for keyword in keywords):
            # categories.append(category)
            return category
    if categories:
        categories.append('기타')
    #return categories
    return '기타'
categorize_content(content)

'납세의무'

In [ ]:
# Exaone으로 메타데이터 추가
enhanced_chunks = []
print("Exaone으로 제목 추출 중...")
for i, chunk in enumerate(documents):
    if i % 10 == 0:
        print(f"   진행: {i}/{len(documents)}")
    content = chunk.page_content
    metadata = chunk.metadata.copy()
    # Exaone으로 제목 추출
    metadata['title'] = extract_title_with_llm(content) # llm으로 제목 뽑기
    metadata['category'] = categorize_content(content) # 카테고리 뽑기
    metadata['chunk_id'] = f"chunk_{i:03d}"
    # 조문 번호 추출
    article_match = re.search(r'제(\d+)조', content)
    if article_match:
        metadata['article'] = int(article_match.group(1))

    enhanced_chunks.append(type(chunk)(
        page_content=content,
        metadata=metadata
    ))
print(f"✅ {len(enhanced_chunks)}개 청크 처리 완료")
enhanced_chunks

Exaone으로 제목 추출 중...
   진행: 0/64
   진행: 10/64
   진행: 20/64
   진행: 30/64


In [129]:
enhanced_chunks[0].metadata

{'source': './data/with_markdown-sample.docx',
 'title': '소득세법: 개인 및 법인의 소득에 따른 적정 과세 규정',
 'category': '납세의무',
 'chunk_id': 'chunk_000',
 'article': 1}

In [ ]:
import pandas as pd

# enhanced_chunks를 DataFrame으로 변환
data = []
for chunk in enhanced_chunks:
    row = {"page_content": chunk.page_content[:10]}
    row.update(chunk.metadata)  # metadata 딕셔너리의 key/value 추가
    data.append(row)

df = pd.DataFrame(data)

# CSV 저장
df.to_csv("./tax_docs/enhanced_chunks.csv", index=False, encoding="utf-8-sig")

### <span style="color: rgb(46, 204, 113);">5. OpenAI 임베딩 모델 설정</span>

In [131]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embedding = OpenAIEmbeddings(
    model=OPENAI_EMBEDDING_MODEL,
    openai_api_key=OPENAI_API_KEY
)

### <span style="color: rgb(46, 204, 113);">6. Pinecone 인덱스 생성 및 vector store(DB) 저장</span>

In [132]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트
pc = Pinecone(api_key=PINECONE_API_KEY)

print('PINECONE_API_KEY의 pinecone index들 :',pc.list_indexes().names())
 
# index = pc.Index(PINECONE_INDEX_NAME)
# index.delete(delete_all=True)
# print("모든 데이터가 삭제되었습니다!")
# 인덱스 생성 여부 확인 및 생성
# if PINECONE_INDEX_NAME not in pc.list_indexes().names():
if not pc.has_index(PINECONE_INDEX_NAME):
    pc.create_index(
        name=PINECONE_INDEX_NAME, 
        dimension=PINECONE_INDEX_DIMENSION, 
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(region=PINECONE_INDEX_REGION,
                            cloud=PINECONE_INDEX_CLOUD
        )
    )
    print(f"인덱스 '{PINECONE_INDEX_NAME}' 생성 완료")
else:
    print(f"인덱스 '{PINECONE_INDEX_NAME}'가 이미 존재합니다.")

print('PINECONE_API_KEY의 pinecone index들 :',pc.list_indexes().names())

PINECONE_API_KEY의 pinecone index들 : ['tax-index', 'reg-retrieval']
인덱스 'better-reg-index' 생성 완료
PINECONE_API_KEY의 pinecone index들 : ['tax-index', 'better-reg-index', 'reg-retrieval']


In [133]:

from langchain_pinecone import PineconeVectorStore
# Pinecone 벡터 스토어 연결
vector_database = PineconeVectorStore.from_documents(
                documents=enhanced_chunks,
                embedding=embedding,
                index_name=PINECONE_INDEX_NAME
            )
print("백터DB 저장")

백터DB 저장


In [134]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
keyword_dict  = [
    "사람을 나타내는 표현 -> 거주자",
    "직장인 -> 근로소득이 있는 거주자", 
    "월급쟁이 -> 근로소득이 있는 거주자",
    "회사원 -> 근로소득이 있는 거주자",
    "연봉 -> 총급여액",
    "월급 -> 근로소득",
    "세금 -> 소득세",
    "공제받다 -> 공제를 적용받다",
    "얼마나 내야하나 -> 세액은 얼마인가",
    "계산해줘 -> 계산하면 얼마인가"
]
prompt = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 
사용자의 질문을 변경해 주세요. 만약 변경할 필요가 없을경우, 사용자의 질문을 변경하지 않아도 됩니다.
그런 경우에는 질문만 리턴해 주세요.
사전: {keyword_dict}
질문: {{question}}
""")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,
                api_key=OPENAI_API_KEY)
keyword_chain = prompt | llm | StrOutputParser()
keyword_chain.invoke({"question":"연봉 5000만원인 회사원의 소득세는 얼마예요?"})

'총급여액 5000만원인 근로소득이 있는 거주자의 소득세는 얼마예요?'

In [141]:
categorize_content("연봉 5000만원인 회사원의 소득세는 얼마예요")

'근로소득'

In [ ]:
# 한국 소득세 전용 프롬프트
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 최고의 한국 소득세 전문가입니다"),
    ("human", """다음과 같은 검색된 문맥을 사용하여 질문에 답하세요. 
답을 모르면 모른다고 말하세요. 
사용자의 질문에 정확하고 자세하게 답변해 주세요.

질문: {question}
문맥: {context}
답변:""")
])
retriever = vector_database.as_retriever(
  search_kwargs={
                "k": 4,
                "filter": {
                    "category": {"$in": ["세율계산", "근로소득", "공제감면"]} # 3개의 카테고리만 검색
                    # title에 특정 키워드가 포함된 문서만 검색(비추. 제목은 rerank에 많이 쓰임 20개쯤 유사한 것을 뽑다 제목과 유사도가 높은 것으로 다시 rerank함)
                    #{"title": {"$regex": f"({'|'.join(title_keywords)})"}} if title_keywords else {}
                }
            }
)
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    chain_type_kwargs={"prompt": prompt_template}
)
full_chain = {"query": keyword_chain} | qa_chain
results = full_chain.invoke({"question":"연봉 5000만원인 회사원의 소득세는 얼마예요?"})

In [136]:
print(results['result'])

총급여액 5,000만원인 근로소득이 있는 거주자의 소득세를 계산하기 위해서는 소득세율과 세액공제를 고려해야 합니다. 

2023년 기준으로 한국의 근로소득세율은 다음과 같습니다:

1. 1,200만원 이하: 6%
2. 1,200만원 초과 ~ 4,600만원 이하: 15%
3. 4,600만원 초과 ~ 8,800만원 이하: 24%
4. 8,800만원 초과 ~ 1억5천만원 이하: 35%
5. 1억5천만원 초과: 38%

총급여액 5,000만원에 대한 소득세를 계산해보면:

1. 1,200만원에 대한 세금: 1,200만원 × 6% = 72만원
2. 1,200만원 초과 4,600만원까지 (3,400만원)에 대한 세금: 3,400만원 × 15% = 510만원
3. 4,600만원 초과 5,000만원까지 (400만원)에 대한 세금: 400만원 × 24% = 96만원

이제 이 세액을 모두 합산합니다:

- 72만원 + 510만원 + 96만원 = 678만원

따라서, 총급여액 5,000만원인 근로소득이 있는 거주자의 소득세는 약 678만원입니다. 

단, 이 계산은 기본적인 세율만을 적용한 것이며, 실제 세액은 각종 세액공제나 추가적인 소득에 따라 달라질 수 있습니다.


In [137]:
# 관련 문서 미리보기
retriever = vector_database.as_retriever(search_kwargs={"k": 3})
docs = retriever.get_relevant_documents("연봉 5000만원인 직장인의 소득세는 얼마예요")
print("🔍 검색된 관련 문서:")
for i, doc in enumerate(docs):
    title = doc.metadata.get('title', '제목없음')
    category = doc.metadata.get('category', '기타')
    print(f"   {i+1}. [{category}] {title}")

🔍 검색된 관련 문서:
   1. [근로소득] 소득세 감면 및 공제 조항: 해외근무 급여, 국가유공자 지원금, 전직대통령 연금, 군인군무원 급여, 전사 급여,
   2. [세율계산] 소득세 납세 범위: 농업소득, 주택임대소득제한 조건, 전통주 제조소득, 일부 임업소득, 어업소득, 특수 복무
   3. [근로소득] 공적연금 및 기타소득 과세 범위 공적연금 및 관련 보상금 국가보훈 및 특별지원금


In [139]:
results = full_chain.invoke({"question":"회사원이 받을 수 있는 공제는?"})
print(results['result'])

근로소득이 있는 거주자가 받을 수 있는 공제는 다음과 같습니다:

1. **공익신탁법에 따른 공익신탁의 이익**: 공익신탁에서 발생하는 이익은 근로소득에서 공제될 수 있습니다.

2. **사업소득**: 특정 조건을 충족하는 사업소득도 공제 대상이 될 수 있습니다. 예를 들어, 농업 소득, 주택 임대 소득(특정 기준 이하), 전통주 제조 소득 등이 포함됩니다.

3. **근로소득과 퇴직소득**: 다음과 같은 소득이 공제될 수 있습니다:
   - 복무 중인 병이 받는 급여
   - 법률에 따라 동원된 사람이 받는 급여
   - 산업재해보상보험법에 따른 각종 급여
   - 고용보험법에 따른 실업급여, 육아휴직 급여 등
   - 국민연금법에 따른 반환일시금 및 사망일시금
   - 외국정부 또는 국제기관에서 근무하는 경우의 급여
   - 복리후생적 성질의 급여 등

4. **연금소득**: 특정 연금소득도 공제 대상이 될 수 있으며, 이는 공적연금 관련법에 따라 받는 유족연금, 장애연금 등이 포함됩니다.

이 외에도 다양한 조건에 따라 추가적인 공제가 가능할 수 있습니다. 각 공제의 세부 사항은 관련 법령 및 대통령령에 따라 달라질 수 있으므로, 구체적인 상황에 따라 전문가의 상담을 받는 것이 좋습니다.


In [140]:
categorize_content("회사원이 받을 수 있는 공제는?")

'공제감면'